In [76]:
import numpy as np
from sklearn.datasets import fetch_lfw_people
import pandas as pd

In [77]:
lfw_people = fetch_lfw_people(min_faces_per_person=40, resize=1.0 , color=True)

In [78]:
X = lfw_people.images 
target = lfw_people.target

W = X.shape[1]
H = X.shape[2]

IMAGE_SIZE = [W, H]

In [79]:
shape = lfw_people.target_names.shape[0]

In [80]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, 
                                                    target, 
                                                    test_size=0.20,
                                                    stratify= target,
                                                    random_state=42)


In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
#from keras.applications.vgg16 import preprocess_input
#from keras.preprocessing import image
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from keras.models import Sequential

In [82]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3] , weights='imagenet', include_top=False)

for layer in vgg.layers:
  layer.trainable = False

In [83]:
x = Flatten()(vgg.output)

prediction = Dense(shape, activation='softmax')(x)

model = Model(inputs=vgg.input, outputs=prediction)

model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 125, 94, 3)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 125, 94, 64)    │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 125, 94, 64)    │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 62, 47, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 62, 47, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 62, 47, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 31, 23, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 31, 23, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 31, 23, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 31, 23, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 15, 11, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 15, 11, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 15, 11, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 15, 11, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 7, 5, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 7, 5, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 7, 5, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 7, 5, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 3, 2, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_10 (Flatten)            │ (None, 3072)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 19)             │        58,387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,773,075 (56.35 MB)

 Trainable params: 58,387 (228.07 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [92]:
model.compile(optimizer='adam', 
              loss="sparse_categorical_crossentropy", 
              metrics=["sparse_categorical_accuracy"])

In [93]:
model.fit(x=x_train, y=y_train, validation_split=0.2 ,epochs=200)

Epoch 1/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 65s 2s/step - loss: 1.7180 - sparse_categorical_accuracy: 0.5056 - val_loss: 1.5404 - val_sparse_categorical_accuracy: 0.5385
Epoch 2/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 59s 2s/step - loss: 1.1867 - sparse_categorical_accuracy: 0.6715 - val_loss: 1.2939 - val_sparse_categorical_accuracy: 0.6154
Epoch 3/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 59s 2s/step - loss: 0.8996 - sparse_categorical_accuracy: 0.7914 - val_loss: 1.1300 - val_sparse_categorical_accuracy: 0.6856
Epoch 4/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 60s 2s/step - loss: 0.7857 - sparse_categorical_accuracy: 0.8143 - val_loss: 0.9963 - val_sparse_categorical_accuracy: 0.7559
Epoch 5/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 61s 2s/step - loss: 0.6280 - sparse_categorical_accuracy: 0.8879 - val_loss: 0.9131 - val_sparse_categorical_accuracy: 0.7759
Epoch 6/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 61s 2s/step - loss: 0.5493 - sparse_categorical_accuracy: 0.9138 - val_loss: 0.8548 - val_sparse_categorical_accuracy: 0.8094
Epoch 7/20

In [94]:
pred = model.predict(x_test)

12/12 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step


In [101]:
pred_max = [np.argmax(v) for v in pred ]


In [109]:
from sklearn.metrics import classification_report ,confusion_matrix

print(classification_report(y_test , pred_max))

              precision    recall  f1-score   support

           0       1.00      0.88      0.93        16
           1       0.57      0.50      0.53         8
           2       0.94      0.96      0.95        47
           3       0.88      0.92      0.90        24
           4       0.81      0.95      0.88       106
           5       0.90      0.82      0.86        22
           6       0.75      0.67      0.71         9
           7       0.75      0.64      0.69        14
           8       0.83      0.91      0.87        11
           9       1.00      0.82      0.90        11
          10       0.70      0.88      0.78         8
          11       0.86      0.55      0.67        11
          12       0.91      0.83      0.87        12
          13       0.83      0.62      0.71         8
          14       0.88      0.88      0.88         8
          15       0.83      0.50      0.62        10
          16       1.00      0.80      0.89        10
          17       0.84    